# Download fcf data for aso rasters 
This notebook contains functions to download and extract fractional forest cover (fcf) data for ASO rasters

In [1]:
import sys
import os
from os.path import basename, exists, expanduser, join
import shutil
import shapely
import rioxarray as rxr
import xarray as xr
import time
import gzip
from urllib.request import urlretrieve
import matplotlib.pyplot as plt
import glob
import rasterio as rio

## Download fcf data

In [2]:
def url_download(url, out_fp, overwrite = False):
    # check if file already exists
    if not exists(out_fp) or overwrite == True:
            urlretrieve(url, out_fp)
    # if already exists. skip download.
    else:
        print('file already exists, skipping')

In [3]:
def download_fcf(out_fp):
    # this is the url from Lievens et al. 2021 paper
    fcf_url = 'https://zenodo.org/record/3939050/files/PROBAV_LC100_global_v3.0.1_2019-nrt_Tree-CoverFraction-layer_EPSG-4326.tif'
    # download just forest cover fraction to out file
    url_download(fcf_url, out_fp)

In [4]:
fcf_path ='../data/tmp/fcf_global.tif'
download_fcf(fcf_path)

## extract fcf data for aso rasters

In [5]:
def fcf_for_aso(aso_raster_fn, fcf_path):
    aso_raster = rxr.open_rasterio(aso_raster_fn).squeeze()
    aso_raster = aso_raster.where(aso_raster>=0, drop=True)
    aso_raster = aso_raster.interpolate_na(dim='x')
    
    # open as dataArray and return
    fcf = rxr.open_rasterio(fcf_path)

    # clip first to avoid super long reproject processes
    fcf = fcf.rio.clip_box(*aso_raster.rio.transform_bounds("EPSG:4326"))
    # reproject FCF to match dataset
    fcf = fcf.rio.reproject_match(aso_raster, resampling=rio.enums.Resampling.bilinear)
    # remove band dimension as it only has one band
    fcf = fcf.squeeze('band')
    # if max is greater than 1 set to 0-1
    if fcf.max() >= 1:
        fcf = fcf / 100
    
    assert fcf.max() <= 1, "Forest cover fraction must be bounded 0-1"
    assert fcf.min() >= 0, "Forest cover fraction must be bounded 0-1"
    
    fcf.to_netcdf(f'../data/fcf/fcf_for_{aso_raster_fn.split("/")[-1][:-4]}.nc')

    #return fcf, aso_raster

In [6]:
# fn = '/home/jovyan/deep-snow/data/ASO/ASO_50m_SD_cleaned/utm10n/ASO_50M_SD_SacramentoMcCloud_20230209_clean.tif'

In [7]:
# fcf_da, aso = fcf_for_aso(fn)

In [8]:
# f, ax = plt.subplots(1, 2, figsize = (10, 10))
# ax[0].imshow(fcf_da)
# ax[0].set_aspect('equal')
# ax[1].imshow(aso, vmin=0, vmax=2, cmap = "Blues")
# ax[1].set_aspect('equal')
# plt.tight_layout()

In [9]:
def fcf_for_aso_all(dir_path, fcf_path):
    raster_paths = glob.glob(f'{dir_path}/*/ASO_50M_SD*.tif')
    for i, path in enumerate(raster_paths):
        print(f'----\nworking on {path.split("/")[-1]}, {i+1}/{len(raster_paths)}\n----')

        fcf_for_aso(path, fcf_path)

In [10]:
dir_path = '/home/jovyan/deep-snow/data/ASO/ASO_50m_SD_cleaned'
fcf_for_aso_all(dir_path, fcf_path)

----
working on ASO_50M_SD_BlueRiver_20230529_clean.tif, 1/252
----
----
working on ASO_50M_SD_Animas_20210419_clean.tif, 2/252
----
----
working on ASO_50M_SD_EastRiver_20230523_clean.tif, 3/252
----
----
working on ASO_50M_SD_EastRiver_20220518_clean.tif, 4/252
----
----
working on ASO_50M_SD_SouthPlatte_20230416_clean.tif, 5/252
----
----
working on ASO_50M_SD_USCOCM_20190407_clean.tif, 6/252
----
----
working on ASO_50M_SD_TaylorAndLottis_20230523_clean.tif, 7/252
----
----
working on ASO_50M_SD_USCOGE_20180331_clean.tif, 8/252
----
----
working on ASO_50M_SD_BlueRiver_20190624_clean.tif, 9/252
----
----
working on ASO_50M_SD_BlueRiver_20230416_clean.tif, 10/252
----
----
working on ASO_50M_SD_WindyGap_20230416_clean.tif, 11/252
----
----
working on ASO_50M_SD_RoaringFork_20230528_clean.tif, 12/252
----
----
working on ASO_50M_SD_USCOBR_20190624_clean.tif, 13/252
----
----
working on ASO_50M_SD_USCOFR_20210319_clean.tif, 14/252
----
----
working on ASO_50M_SD_Blue_20220526_clean.ti